Self Attention

In [3]:
import numpy as np
import math

L,d_k,d_v = 4,8,8

q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)


In [6]:
print("Q",q)
print("K",k)
print("V",v)

Q [[ 0.1541723  -0.51909703 -0.51559703 -1.69764132  0.2017234   0.31568354
   0.20683006 -2.03790404]
 [-0.66185213  0.46778718  0.63934674  4.10622944  1.4071622   1.47155435
  -0.95312512  0.15822869]
 [ 0.1619455   1.06376764  0.48680381 -0.76430452  0.78177303  0.48341978
   1.46749863 -0.95412759]
 [-0.68656016 -0.29716108 -1.37064051  0.18536215  1.21279477 -1.9587999
  -0.33737227 -0.72329303]]
K [[-0.00310336  0.3601577   1.29680782 -0.14949353  1.96017698 -0.35329291
  -0.24717491  0.81741633]
 [ 0.62522138  0.93320564  0.27947318 -0.67914129 -1.30924603 -2.00512315
   0.76153716 -0.48753414]
 [ 0.46146039  1.48778526  0.10413151 -0.9730548  -0.06406295  2.25118308
  -0.02425819  0.05997752]
 [-1.21814955 -1.10212295  1.83493776 -0.32438465  0.19732738 -1.54649501
   0.35019345  0.91052017]]
V [[ 0.82740481 -1.2570743  -1.26607422 -0.05761555 -0.11865103 -0.99178985
   0.57169046 -0.01855821]
 [-0.83945929  0.73554347  0.24768139  0.36816736  1.03298549 -0.55985484
  -0.79336

In [9]:
np.matmul(q,k.T)

array([[-2.03533354,  0.87477695,  1.46754033, -2.24261553],
       [ 2.98911081, -8.18324348, -0.28325923, -2.05594802],
       [ 1.34714751,  1.33896022,  3.39714972, -1.17667828],
       [ 0.65141893,  1.21998797, -5.60453629,  1.08054574]])

In [10]:
q.var(),k.var(),np.matmul(q,k.T).var()

(1.4068255057676113, 1.0107596455063963, 8.75429258008508)

In [11]:
scaled = np.matmul(q,k.T)/np.sqrt(d_k)

In [12]:
scaled.var()

1.0942865725106348

II Masking

In [14]:
mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [17]:
mask[mask==0] = -np.infty
mask[mask==1] = 0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [18]:
scaled + mask

array([[-0.71959907,        -inf,        -inf,        -inf],
       [ 1.05681026, -2.89321348,        -inf,        -inf],
       [ 0.47628857,  0.47339393,  1.2010738 ,        -inf],
       [ 0.23031137,  0.43133088, -1.98150281,  0.38203061]])

In [22]:
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T


In [23]:
attention = softmax(scaled+mask)

In [24]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.98110948, 0.01889052, 0.        , 0.        ],
       [0.24622068, 0.24550899, 0.50827033, 0.        ],
       [0.28604251, 0.34972948, 0.03132217, 0.33290584]])

In [25]:
self_att = np.matmul(attention,v)
self_att

array([[ 0.82740481, -1.2570743 , -1.26607422, -0.05761555, -0.11865103,
        -0.99178985,  0.57169046, -0.01855821],
       [ 0.79591688, -1.21943271, -1.23747859, -0.04957229, -0.09689602,
        -0.98363037,  0.54590385, -0.0479059 ],
       [-0.06239229, -0.14977432,  0.32802959,  0.72268714,  0.0190513 ,
        -1.3516001 , -0.47245347,  0.39203353],
       [-0.23148517, -0.04147487, -0.06467622,  0.72297739,  0.67371657,
        -0.40382029, -0.2178588 , -0.29682661]])

In [30]:
def softmax(x):
    return(np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

def self_atten(q,k,v,mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q,k.T)
    if mask is not None:
        scaled += mask
    attention = softmax(scaled)
    V = np.matmul(attention,v)
    return V , attention


In [32]:
import numpy as np
import math

L,d_k,d_v = 4,8,8

q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)

val,att = self_atten(q,k,v)
print(att)
print(val)

[[4.90710760e-03 9.14721889e-02 6.59366208e-06 9.03614110e-01]
 [1.99369558e-01 2.98664225e-01 1.39184684e-01 3.62781532e-01]
 [3.62516394e-01 1.03388724e-01 1.33353398e-01 4.00741484e-01]
 [1.05395565e-02 6.78533333e-01 1.73272255e-01 1.37654856e-01]]
[[-1.35011073 -0.20092601 -1.46398351 -0.91852437  1.82891154  0.40058442
   0.76431602 -1.25207882]
 [-0.6965227  -0.32949465 -0.50889719 -0.27384133  0.94884947 -0.09032792
   0.87389179 -1.05204293]
 [-1.29030718 -0.1478759  -0.53932545 -0.00756225  0.76453083  0.0758605
   0.83501292 -1.1657984 ]
 [ 0.46840937 -0.66072792 -0.14790408 -0.46742204  0.95479584 -0.53074652
   0.99299288 -0.8198303 ]]
